<a href="https://colab.research.google.com/github/Arthur-L-R/Agentes-de-IA/blob/main/Imers%C3%A3o_Agentes_de_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 01

In [3]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

Importação da API Key

In [4]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com o Google Gemini

In [5]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0.0,
    api_key = GOOGLE_API_KEY
)

In [6]:
resp_test = llm.invoke("Quantos dias de férias eu tenho da empresa?")
print(resp_test.content)

Não tenho como saber essa informação, pois ela é **pessoal e específica** da sua relação de trabalho com a empresa.

Para ter certeza sobre quantos dias de férias você tem direito, você deve consultar:

1.  **Seu Contrato de Trabalho:** Geralmente, as condições de férias estão especificadas lá.
2.  **Departamento de Recursos Humanos (RH) ou Pessoal da Sua Empresa:** Eles são a fonte mais precisa e atualizada sobre seu saldo de férias.
3.  **Manual de Políticas da Empresa:** Muitas empresas têm um documento que detalha as regras de férias.
4.  **Seu Gestor Direto:** Ele pode te orientar ou direcionar para o RH.
5.  **Seu Holerite/Contracheque:** Às vezes, o saldo de dias de férias é informado no documento.

**Informações Gerais (válidas no Brasil, por exemplo):**

*   **Período Aquisitivo:** No Brasil, após cada período de 12 meses de trabalho (período aquisitivo), o empregado tem direito a 30 dias corridos de férias.
*   **Proporcional:** Se você ainda não completou 12 meses de trabalh

In [7]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Lopes Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [8]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)

In [9]:
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0.0,
    api_key = GOOGLE_API_KEY
)

In [10]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content = TRIAGEM_PROMPT),
      HumanMessage(content = mensagem)
  ])

  return saida.model_dump()

In [11]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem em Vigário Geral?"]

In [12]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Respota: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Respota: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Respota: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Respota: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem em Vigário Geral?
 -> Respota: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['contexto_politica']}



# Aula 02

In [13]:
!pip install -q --upgrade langchain_community==0.3.26 faiss-cpu langchain-text-splitters pymupdf

In [14]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("/content/").glob("*.pdf"):
  try:
    loader = PyMuPDFLoader(str(n))
    docs.extend(loader.load())
    print(f"Carregado com sucesso arquivo {n.name}")
  except Exception as e:
    print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

Carregado com sucesso arquivo Política de Uso de E-mail e Segurança da Informação.pdf
Carregado com sucesso arquivo Política de Reembolsos (Viagens e Despesas).pdf
Carregado com sucesso arquivo Políticas de Home Office.pdf
Total de documentos carregados: 3


In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 30)

chunks = splitter.split_documents(docs)

In [ ]:
for chunk in chunks:
  print(chunk)
  print("\n")

In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model = "models/gemini-embedding-001",
    google_api_key = GOOGLE_API_KEY
)

In [20]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type = "similarity_score_threshold",
                                     search_kwargs = {"score_threshold":0.3, "k": 4})

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Lopes Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {pergunta}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [27]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [34]:
from pickle import FALSE
def perguntar_politica_RAG(pergunta: str) -> Dict:
  docs_relacionados = retriever.invoke(pergunta)

  if not docs_relacionados:
    return {"answer": "Não sei.",
            "citacoes": [],
            "contexto_encontrado": False}

  answer = document_chain.invoke({"pergunta": pergunta,
                                  "context": docs_relacionados})

  txt = (answer or "").strip()

  if txt.rstrip(".!?") == "Não sei":
    return {"answer": "Não sei.",
            "citacoes": [],
            "contexto_encontrado": False}

  return {"answer": txt,
          "citacoes": formatar_citacoes(docs_relacionados, pergunta),
          "contexto_encontrado": True}

In [23]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem em Vigário Geral?"]

In [35]:
for msg_teste in testes:
  resposta = perguntar_politica_RAG(msg_teste)
  print(f"PERGUNTA: {msg_teste}")
  print(f"RESPOSTA: {resposta['answer']}")
  if resposta['contexto_encontrado']:
    print("CITAÇÕES:")
    for c in resposta['citacoes']:
      print(f"  - Documento: {c['documento']}")
      print(f"    Página: {c['pagina']}")
      print(f"    Trecho: {c['trecho']}")
    print("\n")

PERGUNTA: Posso reembolsar a internet?
RESPOSTA: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
CITAÇÕES:
  - Documento: Política de Reembolsos (Viagens e Despesas).pdf
    Página: 1
    Trecho: lsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.​ 4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.​
  - Documento: Políticas de Home Office.pdf
    Página: 1
    Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de


PERGUNTA: Quero mais 5 dias de trabalho remoto. Como faço?
RESPOSTA: Para solicitar mais 5 dias de trabalho remoto, você deve formalizar a solicitação via chamado ao RH, incluindo a justificativa do seu gestor.
CITAÇÕES:
  - Documento: Políticas de Home Office.pdf
    Página